--

**使用Keras和TensorFlow Hub对电影评论进行文本分类**

此教程本会将文本形式的影评分为“正面”或“负面”影评。这是一个二元分类（又称为两类分类）的示例，也是一种重要且广泛适用的机器学习问题。

本教程演示了使用TensorFlow Hub和Keras进行迁移学习的基本应用。

数据集使用 IMDB 数据集，其中包含来自互联网电影数据库 https://www.imdb.com/ 的50000 条影评文本。我们将这些影评拆分为训练集（25000 条影评）和测试集（25000 条影评）。训练集和测试集之间达成了平衡，意味着它们包含相同数量的正面和负面影评。

此教程使用tf.keras，一种用于在 TensorFlow 中构建和训练模型的高阶 API，以及TensorFlow Hub，一个用于迁移学习的库和平台。

In [3]:
from __future__ import absolute_import, division, print_function, unicode_literals

import numpy as np

import tensorflow as tf

import tensorflow_hub as hub
import tensorflow_datasets as tfds

print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("Hub version: ", hub.__version__)
print("GPU is", "available" if tf.test.is_gpu_available() else "NOT AVAILABLE")

Version:  2.0.0
Eager mode:  True
Hub version:  0.6.0
GPU is NOT AVAILABLE


# 1. 下载 IMDB 数据集
TensorFlow数据集上提供了IMDB数据集。以下代码将IMDB数据集下载到您的机器：

In [6]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

# 将训练集分成60％和40％，因此我们最终会得到15,000个训练样本，10,000个验证样本和25,000个测试样本。
train_validation_split = tfds.Split.TRAIN.subsplit([6, 4])

(train_data, validation_data), test_data = tfds.load(
    name="imdb_reviews", 
    split=(train_validation_split, tfds.Split.TEST),
    as_supervised=True)

Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Dataset imdb_reviews downloaded and prepared to /Users/zhouwencheng/tensorflow_datasets/imdb_reviews/plain_text/0.1.0. Subsequent calls will reuse this data.


# 2. 探索数据

我们花点时间来了解一下数据的格式，每个样本表示电影评论和相应标签的句子，该句子不以任何方式进行预处理。每个标签都是整数值 0 或 1，其中 0 表示负面影评，1 表示正面影评。

In [7]:
train_examples_batch, train_labels_batch = next(iter(train_data.batch(10)))
train_examples_batch

<tf.Tensor: id=219, shape=(10,), dtype=string, numpy=
array([b"As a lifelong fan of Dickens, I have invariably been disappointed by adaptations of his novels.<br /><br />Although his works presented an extremely accurate re-telling of human life at every level in Victorian Britain, throughout them all was a pervasive thread of humour that could be both playful or sarcastic as the narrative dictated. In a way, he was a literary caricaturist and cartoonist. He could be serious and hilarious in the same sentence. He pricked pride, lampooned arrogance, celebrated modesty, and empathised with loneliness and poverty. It may be a clich\xc3\xa9, but he was a people's writer.<br /><br />And it is the comedy that is so often missing from his interpretations. At the time of writing, Oliver Twist is being dramatised in serial form on BBC television. All of the misery and cruelty is their, but non of the humour, irony, and savage lampoonery. The result is just a dark, dismal experience: the story p

In [8]:
train_labels_batch

<tf.Tensor: id=220, shape=(10,), dtype=int64, numpy=array([1, 1, 1, 1, 1, 1, 0, 1, 1, 0])>

# 3. 构建模
神经网络通过堆叠层创建而成，这需要做出三个架构方面的主要决策：

- 如何表示文字？
- 要在模型中使用多少个层？
- 要针对每个层使用多少个隐藏单元？

在此示例中，输入数据由句子组成。要预测的标签是0或1。

表示文本的一种方法是将句子转换为嵌入向量。我们可以使用预先训练的文本嵌入作为第一层，这将具有两个优点：

- 我们不必担心文本预处理，
- 我们可以从迁移学习中受益
- 嵌入具有固定的大小，因此处理起来更简单。

对于此示例，我们将使用来自TensorFlow Hub 的预训练文本嵌入模型，名为google/tf2-preview/gnews-swivel-20dim/1.

要达到本教程的目的，还有其他三种预训练模型可供测试：

- google/tf2-preview/gnews-swivel-20dim-with-oov/1 与 google/tf2-preview/gnews-swivel-20dim/1相同，但2.5％的词汇量转换为OOV桶。如果模型的任务和词汇表的词汇不完全重叠，这可以提供帮助。
- google/tf2-preview/nnlm-en-dim50/1 一个更大的模型，具有约1M的词汇量和50个维度。
- google/tf2-preview/nnlm-en-dim128/1 甚至更大的模型，具有约1M的词汇量和128个维度。


In [9]:
embedding = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"
hub_layer = hub.KerasLayer(embedding, input_shape=[], 
                           dtype=tf.string, trainable=True)
hub_layer(train_examples_batch[:3])

URLError: <urlopen error [Errno 60] Operation timed out>